<h1>Kaggle: Houses Prices - Advanced Regression Techniques<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Descripción" data-toc-modified-id="Descripción-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Descripción</a></span></li><li><span><a href="#Carga-de-los-datos" data-toc-modified-id="Carga-de-los-datos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Carga de los datos</a></span></li><li><span><a href="#Análisis-exploratorio" data-toc-modified-id="Análisis-exploratorio-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Análisis exploratorio</a></span></li><li><span><a href="#Preprocesado-de-datos" data-toc-modified-id="Preprocesado-de-datos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocesado de datos</a></span><ul class="toc-item"><li><span><a href="#Categorías" data-toc-modified-id="Categorías-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Categorías</a></span></li><li><span><a href="#Pre-procesado" data-toc-modified-id="Pre-procesado-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Pre-procesado</a></span></li><li><span><a href="#Subconjuntos-de-entrenamiento-y-validación" data-toc-modified-id="Subconjuntos-de-entrenamiento-y-validación-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Subconjuntos de entrenamiento y validación</a></span></li></ul></li><li><span><a href="#Entrenamiento-y-test-del-modelo" data-toc-modified-id="Entrenamiento-y-test-del-modelo-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Entrenamiento y test del modelo</a></span><ul class="toc-item"><li><span><a href="#Random-Forests" data-toc-modified-id="Random-Forests-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Random Forests</a></span></li></ul></li></ul></div>

> Notebook creado para participar en la competición de Kaggle [house-prices-advanced-regression-techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)

## Descripción

"Pídale a un comprador de vivienda que describa la casa de sus sueños, y es probable que no empiece con la altura del techo del sótano o la proximidad a las vías del tren. Pero el conjunto de datos de esta competición demuestra que influye mucho más en los precios que el número de habitaciones.

Con 79 variables explicativas que describen (casi) todas las características de los hogares residenciales en Ames (Iowa), esta competición le desafía a predecir el precio final de cada hogar."

* Descargas + Información completa sobre los datasets: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesRegressor
from IPython.display import display
from sklearn import metrics
import os

In [3]:
set_plot_sizes(12,14,16)

## Carga de los datos

Primero tendremos que descargar los datos desde la [web de Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data), y los dejamos en el subdirectorio /data.

A continuación volcamos la información del fichero en un dataframe de Pandas. Usaremos la función info para ver sus muestras (filas) y sus variables (columnas). De estas últimas obtendremos también el tipo y el número de valores que no son NaN.

In [4]:
df_raw = pd.read_csv(f'data/train.csv', low_memory=False)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

## Análisis exploratorio

Lo primero que haremos siempre después de cargar los datos será echar un primer vistazo a los mismos, para saber con qué estamos tratando. Creamos una función auxiliar (recubriendo a la función display de Python) para visualizar el contenido de un dataframe con un límite preestablecido de filas y columnas, y vemos una muestra de nuestro dataframe. Usamos T para transponer filas por columnas y visualizar mejor los datos en el notebook:

In [5]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        display(df)

display_all(df_raw.head().T)

,0,1,2,3,4
Id,1,2,3,4,5
MSSubClass,60,20,60,70,60
MSZoning,RL,RL,RL,RL,RL
LotFrontage,65,80,68,60,84
LotArea,8450,9600,11250,9550,14260
Street,Pave,Pave,Pave,Pave,Pave
Alley,NaN,NaN,NaN,NaN,NaN
LotShape,Reg,Reg,IR1,IR1,IR1
LandContour,Lvl,Lvl,Lvl,Lvl,Lvl
Utilities,AllPub,AllPub,AllPub,AllPub,AllPub


Nuestra variable objetivo o valor a predecir se trata de la columna 'SalePrice'. El resto serán variables dependientes.

Echamos también un vistazo a los estadísticos básicos para todas las columnas, lo que nos dará un poco de información para conocer el contenido de las columnas o variables dependientes:

In [6]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Id,1460,NaN,NaN,NaN,730.5,421.61,1,365.75,730.5,1095.25,1460
MSSubClass,1460,NaN,NaN,NaN,56.8973,42.3006,20,20,50,70,190
MSZoning,1460,5,RL,1151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LotFrontage,1201,NaN,NaN,NaN,70.05,24.2848,21,59,69,80,313
LotArea,1460,NaN,NaN,NaN,10516.8,9981.26,1300,7553.5,9478.5,11601.5,215245
Street,1460,2,Pave,1454,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alley,91,2,Grvl,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LotShape,1460,4,Reg,925,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LandContour,1460,4,Lvl,1311,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Utilities,1460,2,AllPub,1459,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En este caso no vamos a hacer más análisis exploratorio, ya que no tiene mucho sentido si no pensamos quitar variables.

## Preprocesado de datos

Siempre es importante saber qué tipo de métrica se va a usar en un proyecto para la evaluación del modelo. En este caso Kaggle nos dice que evaluará usando RMSLE (Root Mean Squared Log Error), entre la predicción y el valor real de cada precio (columna SalePrice). Por ello usaremos el logaritmo del precio, y así el error RMSE nos dará directamente el RMLSE:

In [7]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

Para las columnas que contienen las variables dependientes observamos que tenemos una mezcla de datos numéricos y datos categóricos. 

### Categorías

Para las categorías lo que hacemos es convertir las columnas cuyos valores son strings al tipo category de Pandas. De tal forma que para Pandas ya es como si fueran variables numéricas, puesto que internamente asigna a cada string un código numérico.

Primeramente podemos echar un vistazo al número de valores únicos dentro de cada variable:

In [8]:
unique_counts = pd.DataFrame.from_records([(col, df_raw[col].nunique()) 
                                           for col in df_raw.columns if df_raw[col].dtype == 'object'],
                                           columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
unique_counts

,Column_Name,Num_Unique
1,Street,2
2,Alley,2
28,CentralAir,2
5,Utilities,2
7,LandSlope,3
38,PoolQC,3
37,PavedDrive,3
34,GarageFinish,3
21,BsmtQual,4
18,ExterQual,4


Usamos la función `train_cats` para convertir todas las columnas del dataframe de tipo object a tipo category:

In [9]:
train_cats(df_raw)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null category
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null category
Alley            91 non-null category
LotShape         1460 non-null category
LandContour      1460 non-null category
Utilities        1460 non-null category
LotConfig        1460 non-null category
LandSlope        1460 non-null category
Neighborhood     1460 non-null category
Condition1       1460 non-null category
Condition2       1460 non-null category
BldgType         1460 non-null category
HouseStyle       1460 non-null category
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null category
RoofMatl         1460 non-null catego

Además de nuestro objetivo hemos conseguido reducir el tamaño en memoria del dataframe a una cuarta parte del original!

### Pre-procesado

Como hemos podido ver en nuestro primer análisis exploratorio, tenemos un montón de NaN, que no podemos pasar al algoritmo Random Forests. Por suerte contamos con la función `proc_df` de fastai, que nos resuelve varias cuestiones:
 * Ajustar el mapeo entre categorías y números (mediante numericalize, que convierte -1 en 0, 0 en 1, etc) y quedarse únicamente con los valores numéricos.
 * Crear dummies (categorías con pocos valores posibles)
 * Manejar los missing values en columnas numéricas (fix_missing). No hace falta en las categóricas porque Pandas ya los tradujo a -1.
 * Separar la variable dependiente del resto

In [10]:
df, y, nas = proc_df(df_raw, 'SalePrice')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 83 columns):
Id                1460 non-null int64
MSSubClass        1460 non-null int64
MSZoning          1460 non-null int8
LotFrontage       1460 non-null float64
LotArea           1460 non-null int64
Street            1460 non-null int8
Alley             1460 non-null int8
LotShape          1460 non-null int8
LandContour       1460 non-null int8
Utilities         1460 non-null int8
LotConfig         1460 non-null int8
LandSlope         1460 non-null int8
Neighborhood      1460 non-null int8
Condition1        1460 non-null int8
Condition2        1460 non-null int8
BldgType          1460 non-null int8
HouseStyle        1460 non-null int8
OverallQual       1460 non-null int64
OverallCond       1460 non-null int64
YearBuilt         1460 non-null int64
YearRemodAdd      1460 non-null int64
RoofStyle         1460 non-null int8
RoofMatl          1460 non-null int8
Exterior1st       1460 non-null 

In [11]:
df.head().T

,0,1,2,3,4
Id,1,2,3,4,5
MSSubClass,60,20,60,70,60
MSZoning,4,4,4,4,4
LotFrontage,65,80,68,60,84
LotArea,8450,9600,11250,9550,14260
Street,2,2,2,2,2
Alley,0,0,0,0,0
LotShape,4,4,1,1,1
LandContour,4,4,4,4,4
Utilities,1,1,1,1,1


### Subconjuntos de entrenamiento y validación

Kaggle nos proporciona los siguientes ficheros de datos:
 * Train.csv : datos usados para entrenar el modelo
 * Test.csv : datos usados para determinar el ranking final -> Private LeaderBoard

Esto implica que tenemos que usar el dataset de entrenamiento también para validación.

In [12]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()

n_trn = 1300

X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((1300, 83), (1300,), (160, 83))

## Entrenamiento y test del modelo

Una vez pre-procesado el dataset, procedemos con el modelo. 

Primero creamos unas funciones auxiliares para calcular todas las medidas necesarias para la evaluación del modelo:

In [13]:
def rmse(predictions, actuals): 
    return math.sqrt(((predictions - actuals)**2).mean())

def print_score(m):
    print('RMSE for training:   ', rmse(m.predict(X_train), y_train))
    print('RMSE for validation: ', rmse(m.predict(X_valid), y_valid))
    print('R^2 for training:    ', m.score(X_train, y_train))
    print('R^2 for validation:  ', m.score(X_valid, y_valid))
    if hasattr(m, 'oob_score_'): 
        print('OoB error:           ', m.oob_score_)

Creamos también una función para aplicar nuestro modelo a los datos de test, predecir la variable target, y crear el fichero para su envío a Kaggle:

In [14]:
def test_and_submit(m):
    # Cargamos el dataset que nos da Kaggle
    df_raw_test = pd.read_csv(f'data/test.csv', low_memory=False)
    
    # Usamos la función apply_cats de fastai para replicar lo que hicimos con train_cats sobre el dataset de entrenamiento
    apply_cats(df_raw_test, df_raw)
    
    # Aplicamos el pre-procesado para obtener el dataframe de entrada al modelo
    X_test, _, _ = proc_df(df_raw_test, na_dict=nas)
    
    # Aplicamos el modelo a los datos de test para predecir los valores de salida
    y_pred = m.predict(X_test)
    
    # Por último creamos el fichero para enviar a Kaggle!
    submission = pd.DataFrame()
    submission['Id'] = X_test.Id
    submission['SalePrice'] = np.exp(y_pred) # Deshacemos el logaritmo aplicado en el modelo
    submission.to_csv(f'results/submission.csv',index=False)

### Random Forests 
A continuación instanciamos la clase adecuada con el nº de estimadores (árboles de decisión) por defecto, y paralelización de trabajos:

In [15]:
m1 = RandomForestRegressor(n_jobs=-1, n_estimators=10)
%time m1.fit(X_train, y_train)
print_score(m1)

Wall time: 120 ms
RMSE for training:    0.06481027704095792
RMSE for validation:  0.15991490144434978
R^2 for training:     0.9735733141368982
R^2 for validation:   0.8436669318321928


In [16]:
m1b = RandomForestRegressor(n_jobs=-1, n_estimators=50, oob_score=True)
%time m1b.fit(X_train, y_train)
print_score(m1b)

Wall time: 250 ms
RMSE for training:    0.054096564000256725
RMSE for validation:  0.15160114612154946
R^2 for training:     0.9815882822305116
R^2 for validation:   0.8594994736854019
OoB error:            0.8668170310122272


Parece que tenemos sobreajuste por lo que se puede observar en las medidas. Confirmado cuando subimos nuestra salida a Kaggle (error: 0,15)

Vamos a intentar mejorar el sobreajuste tuneando sus parámetros:

In [25]:
m1c = RandomForestRegressor(n_estimators=100, min_samples_leaf=2, max_features=0.5, n_jobs=-1, oob_score=True)
m1c.fit(X_train, y_train)
print_score(m1c)

RMSE for training:    0.062397035200046164
RMSE for validation:  0.14047000895202666
R^2 for training:     0.9755046947577716
R^2 for validation:   0.87937420127091
OoB error:            0.8788169652918847


Por último probamos con Extra Trees para comparar:

In [18]:
m2 = ExtraTreesRegressor(n_estimators=40, min_samples_leaf=5, max_features=0.5, n_jobs=-1, 
                        bootstrap=True, oob_score=True)
m2.fit(X_train, y_train)
print_score(m2)

RMSE for training:    0.11424238661596386
RMSE for validation:  0.14375883843270204
R^2 for training:     0.917887461030325
R^2 for validation:   0.8736596452766656
OoB error:            0.8513420842319446


Finalmente aplicamos el mejor modelo sobre los datos de test:

In [26]:
test_and_submit(m1c)

Hemos mejorado un poco (error: 0,145), pero aún estamos lejos de obtener una posición decente. El próximo paso sería optimizar los parámetros usando grid search.